# Part 2 NLP (25 pts)

The Part 2 is an in-class competition. 

Original Competition: https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge 

Using the file: jigsaw-toxic-comment-classification-challenge.zip

The .zip file contains the Jigsaw Toxic Comment Classification dataset as provided in the original Kaggle competition. 
Data is organized as zip files containing .csv files. The training data is organized by ID, text, and label.


Content Warning: The dataset contains text that may be considered profane, vulgar, or offensive.


Submission Details: part2.ipynb file (Write your comments as markdown) Model and results (10 pts) + class ranking (10 pts) + report (5 pts)

review -

https://colab.research.google.com/github/kwanhong66/PyTorchKaggle/blob/master/Toxic_comment_classification_bert_simple.ipynb#scrollTo=y-G3zxj8zgVM

refernce - 
https://towardsdatascience.com/toxic-comment-classification-using-lstm-and-lstm-cnn-db945d6b7986

https://github.com/shaunak09vb/Toxic-Comment-Classifier-AWS/blob/main/source

# Step 1. Load the dataset

In [1]:
import pandas as pd

In [2]:
base_path = "jigsaw-toxic-comment-classification-challenge"
train_df = pd.read_csv(f'{base_path}/train.csv')

print(f"{train_df.shape = } \n")
train_df.head()

train_df.shape = (159571, 8) 



,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0



# Step 2. Preprocess the data as you see fit


In [64]:
! mkdir model2
! rm -rf wiki-news-300d-1M.vec.zip
! wget  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
! unzip wiki-news-300d-1M.vec.zip
! mv wiki-news-300d-1M.vec model2/
! rm -rf wiki-news-300d-1M.vec.zip


--2024-04-28 10:54:54--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 2600:9000:20a6:b600:13:6e38:acc0:93a1, 2600:9000:20a6:de00:13:6e38:acc0:93a1, 2600:9000:20a6:4e00:13:6e38:acc0:93a1, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|2600:9000:20a6:b600:13:6e38:acc0:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  50.1MB/s    in 12s     

2024-04-28 10:55:06 (52.4 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]

Archive:  wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [ ]:
EMBEDDING_DIMENSION = 300
EMBEDDING_FILE_LOCATION = 'model/wiki-news-300d-1M.vec'
DETECTION_CLASSES = [
    'toxic',
    'severe_toxic',
    'obscene',
    'threat',
    'insult',
    'identity_hate',
    'neutral']

MODEL_LOCATION = 'model/toxicity_classifier.h5'
TOKENIZER_LOCATION = 'model/tokenizer.pickle'

In [3]:
import re
import spacy
import nltk
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import string
from string import ascii_lowercase


from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')



[nltk_data] Downloading package wordnet to /Users/hims/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
import contractions

def remove_contraction_(text):
    try:
        specials = ["’", "‘", "´", "`"]
        for s in specials:
            text = text.replace(s, "'")
        expanded_words = [] 
        for word in text.split():
            try:
                expanded_words.append(contractions.fix(word)) 
            except Exception as e:
                expanded_words.append(word)

        return ' '.join(expanded_words)
    except Exception as e:
        print(f"Error occurred during remove_contraction: {e}")
        print(f"{text = }")
        raise e

        
def remove_contraction(text):
    contraction_mapping = {"ain't": "is not", "aren't": "are not", "can't": "cannot", "'cause": "because",
                           "could've": "could have", "couldn't": "could not",
                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                           "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will",
                           "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
                           "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will",
                           "I'll've": "I will have", "I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will", "i'll've": "i will have", "i'm": "i am",
                           "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
                           "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us",
                           "ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not",
                           "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
                           "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                           "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
                           "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will",
                           "she'll've": "she will have", "she's": "she is", "should've": "should have",
                           "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                           "so's": "so as",
                           "this's": "this is", "that'd": "that would", "that'd've": "that would have",
                           "that's": "that is",
                           "there'd": "there would", "there'd've": "there would have", "there's": "there is",
                           "here's": "here is",
                           "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
                           "they'll've": "they will have",
                           "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                           "we'd": "we would", "we'd've": "we would have", "we'll": "we will",
                           "we'll've": "we will have",
                           "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
                           "what'll've": "what will have",
                           "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
                           "when've": "when have",
                           "where'd": "where did", "where's": "where is", "where've": "where have",
                           "who'll": "who will",
                           "who'll've": "who will have",
                           "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have",
                           "will've": "will have",
                           "won't": "will not", "won't've": "will not have", "would've": "would have",
                           "wouldn't": "would not",
                           "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                           "y'all'd've": "you all would have",
                           "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would",
                           "you'd've": "you would have", "you'll": "you will",
                           "you'll've": "you will have", "you're": "you are", "you've": "you have"}
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in text.split(" ")])
    return text

In [6]:
def clean_repeat_patterns_lower(text,remove_repeat_text=True, remove_patterns_text=True, is_lower=True):
    
    RE_PATTERNS = {
        ' american ':
            [
                'amerikan'
            ],

        ' adolf ':
            [
                'adolf'
            ],


        ' hitler ':
            [
                'hitler'
            ],

        ' fuck':
            [
                '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
                '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
                ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
                '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
                'feck ', ' fux ', 'f\*\*', 'f**k','fu*k',
                'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck'
            ],

        ' ass ':
            [
                '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$',
                '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
                'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s','a55', '@$$'
            ],

        ' ass hole ':
            [
                ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole', 'a**hole'
            ],

        ' bitch ':
            [
                'b[w]*i[t]*ch', 'b!tch',
                'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
                'biatch', 'bi\*\*h', 'bytch', 'b i t c h', 'b!tch', 'bi+ch', 'l3itch'
            ],

        ' bastard ':
            [
                'ba[s|z]+t[e|a]+rd'
            ],

        ' trans gender':
            [
                'transgender'
            ],

        ' gay ':
            [
                'gay'
            ],

        ' cock ':
            [
                '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
                '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
            ],

        ' dick ':
            [
                ' dick[^aeiou]', 'deek', 'd i c k', 'dik'
            ],

        ' suck ':
            [
                'sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'
            ],

        ' cunt ':
            [
                'cunt', 'c u n t'
            ],

        ' bull shit ':
            [
                'bullsh\*t', 'bull\$hit'
            ],

        ' homo sex ual':
            [
                'homosexual'
            ],

        ' jerk ':
            [
                'jerk'
            ],

        ' idiot ':
            [
                'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots'
                                                                                          'i d i o t'
            ],

        ' dumb ':
            [
                '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
            ],

        ' shit ':
            [
                'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t', '$h1t'
            ],

        ' shit hole ':
            [
                'shythole'
            ],

        ' retard ':
            [
                'returd', 'retad', 'retard', 'wiktard', 'wikitud'
            ],

        ' rape ':
            [
                ' raped'
            ],

        ' dumb ass':
            [
                'dumbass', 'dubass'
            ],

        ' ass head':
            [
                'butthead'
            ],

        ' sex ':
            [
                'sexy', 's3x', 'sexuality'
            ],


        ' nigger ':
            [
                'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
            ],

        ' shut the fuck up':
            [
                'stfu', 'st*u'
            ],

        ' pussy ':
            [
                'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses', 'p*ssy'
            ],

        ' faggot ':
            [
                'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
                '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
            ],

        ' mother fucker':
            [
                ' motha ', ' motha f', ' mother f', 'motherucker',
            ],

        ' whore ':
            [
                'wh\*\*\*', 'w h o r e'
            ],
        ' fucking ':
            [
                'f*$%-ing'
            ],
    }
    
    if is_lower:
        text=text.lower()

    if remove_patterns_text:
        for target, patterns in RE_PATTERNS.items():
            for pat in patterns:
                text=str(text).replace(pat, target)

    if remove_repeat_text:
        text = re.sub(r'(.)\1{2,}', r'\1', text) 

    text = str(text).replace("\n", " ")
    text = re.sub(r'[^\w\s]',' ',text)
    text = re.sub('[0-9]',"",text)
    text = re.sub(" +", " ", text)
    text = re.sub("([^\x00-\x7F])+"," ",text)
    return text 



def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"
        u"\U0001F1F2-\U0001F1F4"  
        u"\U0001F1E6-\U0001F1FF"
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# from cleantext import clean

# def remove_special_characters(text):
#     # Use the clean-text library to remove emojis and special characters
#     cleaned_text = clean(text, no_emoji=True, no_special_chars=True)
#     return cleaned_text


def prepare_stopwords_list():
    stopword_list = STOP_WORDS
    potential_stopwords = ['editor', 'reference', 'thank', 'work','find', 'good', 'know', 'like', 
                           'look', 'thing', 'want', 'time', 'list', 'section','wikipedia', 'doe', 
                           'add','new', 'try', 'think', 'write','use', 'user', 'way', 'page']
    for word in potential_stopwords:
        stopword_list.add(word)
    return(stopword_list)


def remove_stop_words(text, remove_stop=True):
    stop_words=prepare_stopwords_list()
    output = ""
    if remove_stop:
        text=text.split(" ")
        for word in text:
            if word not in stop_words:
                output=output + " " + word
    else :
        output=text  
    return str(output.strip()) 
    

def lemmatize(text, lemmatization=True):
    wordnet_lemmatizer = WordNetLemmatizer()
    output=""
    if lemmatization:
        text=text.split(" ")
        for word in text:
            word1 = wordnet_lemmatizer.lemmatize(word, pos = "n")
            word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
            word3 = wordnet_lemmatizer.lemmatize(word2, pos = "a")
            word4 = wordnet_lemmatizer.lemmatize(word3, pos = "r")
            output=output + " " + word4
    else:
        output=text
    return str(output.strip())    


# import spacy

# # Load the English language model
# nlp = spacy.load('en_core_web_sm')

# def lemmatize(text):
#     # Process the text using spaCy
#     doc = nlp(text)
    
#     # Lemmatize each token in the document
#     lemmatized_tokens = [token.lemma_ for token in doc]
    
#     # Join the lemmatized tokens back into a string
#     lemmatized_text = ' '.join(lemmatized_tokens)
    
#     return lemmatized_text


def clean_text(text):
    text=remove_contraction(text)
    text=clean_repeat_patterns_lower(text)
    text=remove_emojis(text)
    text=remove_stop_words(text)
    text=lemmatize(text)
    return text


def clean_text_column(text_column):
    return text_column.apply(lambda x: clean_text(x))



In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

class DataProcessing:
    def __init__(self, EMBEDDING_FILE_LOCATION, DETECTION_CLASSES,
                 max_vocab_size=100000, max_seq_length=100, embedding_dimension=300):
        self.max_vocab_size = max_vocab_size
        self.max_seq_length = max_seq_length
        self.embedding_dimension = embedding_dimension
        self.EMBEDDING_FILE_LOCATION = EMBEDDING_FILE_LOCATION
        self.tokenizer = Tokenizer(num_words=self.max_vocab_size)
        self.word_index = None
        self.embedding_layer = None

    def fit_tokenizer(self, data):
        data['comment_text'] = clean_text_column(data['comment_text'])
        self.tokenizer.fit_on_texts(data['comment_text'].values)
        self.word_index = self.tokenizer.word_index
        list_tokenized_train = self.tokenizer.texts_to_sequences(data['comment_text'].values)
        X_t = pad_sequences(list_tokenized_train, maxlen=self.max_seq_length, padding='post')
        return X_t
        
    def get_tokenizer(self):
        return self.tokenizer

    def get_embedding_layer(self):
        # Load pre-trained embeddings
        embeddings_index_fasttext = {}
        with open(self.EMBEDDING_FILE_LOCATION, encoding='utf-8') as f:
            for line in f:
                values = line.split()
                word = values[0]
                embeddings_index_fasttext[word] = np.asarray(values[1:], dtype='float32')

        # Create embedding matrix
        embedding_matrix_fasttext = np.random.random((len(self.word_index) + 1, self.embedding_dimension))
        for word, i in self.word_index.items():
            embedding_vector = embeddings_index_fasttext.get(word)
            if embedding_vector is not None:
                embedding_matrix_fasttext[i] = embedding_vector

        # Initialize embedding layer
        self.embedding_layer = nn.Embedding(len(self.word_index) + 1, self.embedding_dimension)
        self.embedding_layer.weight = nn.Parameter(torch.Tensor(embedding_matrix_fasttext))
        return self.embedding_layer
    
    

    def tokenize(self, data):
        data['comment_text'] = clean_text_column(data['comment_text'])
        list_tokenized = self.tokenizer.texts_to_sequences(data['comment_text'].values)
        X_t = pad_sequences(list_tokenized, maxlen=self.max_seq_length, padding='post')
        return X_t
    
    
    def tokenize_string(self, text):
        text = clean_text(text)
        print(text)
        list_tokenized = self.tokenizer.texts_to_sequences([text])
        X_t = pad_sequences(list_tokenized, maxlen=self.max_seq_length, padding='post')
        return X_t

    
processor = DataProcessing(EMBEDDING_FILE_LOCATION, DETECTION_CLASSES)
X_t = processor.fit_tokenizer(train_df)

In [8]:
# Convert data to PyTorch tensors

data_tensor = torch.tensor(X_t, dtype=torch.long)
target_tensor = torch.tensor(train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values, dtype=torch.float)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data_tensor, target_tensor, test_size=VALIDATION_SPLIT)

# Create DataLoader for training and validation sets
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Step 3. Utilize a model implementing a Natural Language Processing strategy


In [9]:
class LSTMModel(nn.Module):
    
    def __init__(self, embedding_layer, num_classes, hidden_size=40, dropout=0.1):
        super(LSTMModel, self).__init__()
        self.embedding_layer = embedding_layer
        self.lstm = nn.LSTM(embedding_layer.embedding_dim, hidden_size, batch_first=True)
        self.global_max_pooling = nn.AdaptiveMaxPool1d(1)
        self.dropout1 = nn.Dropout(dropout)
        self.dense1 = nn.Linear(hidden_size, 30)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        self.dense2 = nn.Linear(30, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded_sequences = self.embedding_layer(x)
        lstm_output, _ = self.lstm(embedded_sequences)
        global_max_pooled = self.global_max_pooling(lstm_output.permute(0, 2, 1))
        x = self.dropout1(global_max_pooled.squeeze(2))
        x = self.relu(self.dense1(x))
        x = self.dropout2(x)
        x = self.dense2(x)
        return self.sigmoid(x)

# Step 4. Train your model.


In [10]:
import matplotlib.pyplot as plt

# # Given loss values
# train_losses = [0.0638, 0.0440]
# val_losses = [0.0492, 0.0480]

# Number of epochs

def plot_training_metrics(train_losses, val_losses):
    epochs = range(1, len(train_losses) + 1)

    # Plotting the training and validation loss
    plt.plot(epochs, train_losses, 'b', label='Training Loss')
    plt.plot(epochs, val_losses, 'r', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


In [11]:
from tqdm import tqdm  # Import tqdm for progress tracking
import torch.optim as optim



def train(train_loader, val_loader, model, criterion, optimizer, num_epochs):
    
    print("Started training")
    train_losses, val_losses = [], []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in tqdm(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()

        print(f'Epoch {epoch + 1}/{EPOCHS}, Loss: {running_loss / len(train_loader):.4f}, Val Loss: {val_loss / len(val_loader):.4f}')
        
        train_losses.append(running_loss / len(train_loader))
        val_losses.append(val_loss / len(val_loader))
        
    return train_losses, val_losses, model

        

embedding_layer = processor.get_embedding_layer()
model = LSTMModel(embedding_layer, num_classes=6)

train_losses, val_losses, model = train(train_loader, val_loader, 
                                        model, criterion = nn.BCELoss(), 
                                        optimizer = optim.Adam(model.parameters()), 
                                        num_epochs = EPOCHS)


Started training


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3990/3990 [05:41<00:00, 11.68it/s]


Epoch 1/2, Loss: 0.0634, Val Loss: 0.0460


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3990/3990 [05:42<00:00, 11.66it/s]


Epoch 2/2, Loss: 0.0437, Val Loss: 0.0455


# Step 5. Display the results of your model on the Test dataset by showing the predicted labels against their true labels

In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score



def evaluate(model, loader):
    predicted = []
    actual = []
    flag = False
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            predicted.extend(outputs)
            actual.extend(labels)
            if flag:
                flag = False
                print(inputs)
                print(outputs)
                
    print(predicted[0], '\n', actual[0])

    threshold = 0.5
    predicted_classes = [[1 if val >= threshold else 0 for val in output] for output in predicted]
    precision = []
    recall = []
    f1 = []
    accuracy = []
    roc_auc = []
    for i in range(len(actual[0])):
        precision.append(precision_score([row[i] for row in actual], [row[i] for row in predicted_classes]))
        recall.append(recall_score([row[i] for row in actual], [row[i] for row in predicted_classes]))
        f1.append(f1_score([row[i] for row in actual], [row[i] for row in predicted_classes]))
        accuracy.append(accuracy_score([row[i] for row in actual], [row[i] for row in predicted_classes]))
        roc_auc.append(roc_auc_score([row[i] for row in actual], [row[i] for row in predicted]))

    # Create a DataFrame to display the metrics
    metrics_df = pd.DataFrame({
        'Class': [DETECTION_CLASSES[i] for i in list(range(len(actual[0])))] + ['average'],
        'Precision': precision + [np.mean(precision)],
        'Recall': recall + [np.mean(recall)],
        'F1-score': f1 + [np.mean(f1)],
        'Accuracy': accuracy + [np.mean(accuracy)],
        'ROC AUC': roc_auc + [np.mean(roc_auc)]
    })

    # Display the DataFrame
    print(metrics_df.head(10))
    
    
evaluate(model, val_loader)

tensor([[38344,   793, 38344,  ...,     0,     0,     0],
        [42475,   338,     3,  ...,     0,     0,     0],
        [   61,  9490,  1125,  ...,     0,     0,     0],
        ...,
        [ 1802,   245,   200,  ...,   200,   236, 18700],
        [  907,    59, 11073,  ...,     0,     0,     0],
        [    8,    17,    35,  ...,     0,     0,     0]])
tensor([[1.2926e-03, 5.9411e-06, 3.5313e-04, 8.8784e-06, 7.6399e-05, 1.1987e-05],
        [1.5513e-03, 5.1169e-06, 5.3402e-04, 6.3467e-06, 1.0491e-04, 1.1368e-05],
        [1.2914e-04, 2.0728e-07, 2.3377e-05, 3.2486e-07, 4.1329e-06, 4.6442e-07],
        [5.2545e-04, 1.4099e-06, 7.9261e-05, 3.0255e-06, 2.0635e-05, 3.4574e-06],
        [5.5663e-04, 1.3365e-06, 1.0291e-04, 2.4775e-06, 2.5807e-05, 3.3241e-06],
        [6.0523e-04, 1.1928e-06, 7.1045e-05, 3.3086e-06, 2.8275e-05, 3.9408e-06],
        [3.8661e-01, 2.0375e-03, 2.5883e-02, 1.4279e-02, 9.0902e-02, 2.3587e-02],
        [2.0242e-04, 3.4132e-07, 3.2674e-05, 6.0776e-07, 7.8310e

/Users/hims/anaconda3/envs/python310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hims/anaconda3/envs/python310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


           Class  Precision    Recall  F1-score  Accuracy   ROC AUC
0          toxic   0.833090  0.764037  0.797071  0.963528  0.977809
1   severe_toxic   0.558011  0.305136  0.394531  0.990287  0.990240
2        obscene   0.819129  0.816626  0.817876  0.981357  0.992261
3         threat   0.000000  0.000000  0.000000  0.996835  0.969773
4         insult   0.742055  0.658361  0.697707  0.972740  0.982429
5  identity_hate   0.000000  0.000000  0.000000  0.992229  0.972011
6        average   0.492048  0.424027  0.451197  0.982829  0.980754


In [59]:
df1 = pd.read_csv(f'jigsaw-toxic-comment-classification-challenge/test.csv')
df2 = pd.read_csv(f'jigsaw-toxic-comment-classification-challenge/test_labels.csv')

df_test = pd.merge(df1, df2, on='id', how='inner')

# removing sample with labels equal to -1
df_test = df_test.loc[df_test['toxic'] >= 0]
df_test.reset_index(inplace=True)
df_test = df_test.drop(columns=['index'])
# print(df_test.head())
print("test shape = ", df_test.shape)
# print(df_test.head(10))


# df_test['comment_text'] = clean_text_column(df_test['comment_text'])
X_test = processor.tokenize(df_test)


# Convert data to PyTorch tensors
test_data_tensor = torch.tensor(X_test, dtype=torch.long)
test_target_tensor = torch.tensor(df_test[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values, dtype=torch.float)

test_dataset = TensorDataset(test_data_tensor, test_target_tensor)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

evaluate(model, test_loader)

test shape =  (63978, 8)
                 id  \
0  0001ea8717f6de06   
1  000247e83dcc1211   
2  0002f87b16116a7f   
3  0003e1cccfd5a40a   
4  00059ace3e3e9a53   
5  000663aff0fffc80   
6  000689dd34e20979   
7  000844b52dee5f3f   
8  00091c35fa9d0465   
9  000968ce11f5ee34   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   comment_text  \
0                                                                                                                                                                                                             

/Users/hims/anaconda3/envs/python310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hims/anaconda3/envs/python310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


           Class  Precision    Recall  F1-score  Accuracy   ROC AUC
0          toxic   0.549460  0.843678  0.665501  0.919269  0.964756
1   severe_toxic   0.428571  0.465940  0.446475  0.993373  0.988913
2        obscene   0.604373  0.793823  0.686263  0.958126  0.978355
3         threat   0.000000  0.000000  0.000000  0.996702  0.965655
4         insult   0.616347  0.618325  0.617334  0.958939  0.967661
5  identity_hate   0.000000  0.000000  0.000000  0.988871  0.967116
6        average   0.366459  0.453628  0.402596  0.969213  0.972076


In [60]:
def get_labels(text):
    seq = processor.tokenize_string(text)
    # print(seq)
    predicted = model(torch.tensor(seq))
    # print(predicted)
    predicted_classes = [[1 if val >= 0.5 else 0 for val in output] for output in predicted]
    # print(predicted_classes[0])
    return [DETECTION_CLASSES[i] for i, x in enumerate(predicted_classes[0]) if x == 1]


In [63]:
df1 = pd.read_csv(f'jigsaw-toxic-comment-classification-challenge/test.csv')
df2 = pd.read_csv(f'jigsaw-toxic-comment-classification-challenge/test_labels.csv')

df_test = pd.merge(df1, df2, on='id', how='inner')

# removing sample with labels equal to -1
df_test = df_test.loc[df_test['toxic'] >= 0]
df_test.reset_index(inplace=True)
df_test = df_test.drop(columns=['index'])


# Filter rows where sum of labels is greater than 0
filtered_df = df_test[(df_test.iloc[:, 2:].sum(axis=1) > 1)].copy()

# Concatenate labels into a single string using apply and lambda
filtered_df['labels'] = filtered_df.apply(lambda row: ','.join(row.index[2:][row[2:] > 0]), axis=1)

# Create new DataFrame with comment_text and labels
selected_df = filtered_df[['comment_text', 'labels']].head(5)
selected_df.rename(columns={'labels': 'actual_labels'}, inplace=True)
pd.set_option('display.max_colwidth', None)
selected_df['predicted_label'] = selected_df['comment_text'].apply(get_labels)


selected_df


dj robinson gay hell suck dick
fuck anti semitic cunt
arrogant self serve immature idiot right
suck university nicknameless ncaa history happen
argh random idiot delete japan article


,comment_text,actual_labels,predicted_label
24,DJ Robinson is gay as hell! he sucks his dick so much!!!!!,"toxic,obscene,insult,identity_hate","[toxic, obscene, insult]"
29,":Fuck off, you anti-semitic cunt. |","toxic,obscene,insult","[toxic, obscene, insult]"
40,"::No, he is an arrogant, self serving, immature idiot. Get it right.","toxic,obscene,insult","[toxic, insult]"
106,"::::Well, it sucks to have a university to be nicknameless. And it's the first time in NCAA history that it has happened. /","toxic,obscene","[toxic, obscene]"
107,== Argh == \n\n Some random idiot deleted the whole Japan article <.,"toxic,obscene,insult",[toxic]
